In [1]:
import pandas as pd

In [24]:
#we just want to collect the montly number of checkouts, so we only use the the material type, checkout year, checkout month, and checkouts columns
df=pd.read_csv('data/Checkouts_by_Title.csv',usecols=['MaterialType','CheckoutYear','CheckoutMonth','Checkouts'])
#If we want to do tests on smaller datasets to make sure the code is working, we could just sample some of the rows at this step

In [26]:
#get rid of anything with missing values
df=df.dropna()

In [39]:
#We want need to conver the Checkout Year and Checkout Month into a date. This needs a day to work, so we say that the first of the month is our day

df['day']=1

df=df.rename(columns={'CheckoutYear':'year','CheckoutMonth':'month'})
#to_datetime needs to have a column named year and a column named month
df['date']=pd.to_datetime( df[['year','month','day']])

In [ ]:
#We get the first and last months in our dataset
display(df['date'].max())
display(df['date'].min())

In [44]:
#this gives us a pandas series covering the months that we care about.
#The start is df['date'].min() and the end is df['date'].max(), it would be more proper to have those as variables rather than hardcoded
months_of_interest=pd.date_range(start='2005-04-01', end='2024-08-01',freq='MS')

In [46]:
#this makes a data frame where the first two is our months that we care about
dfmonths=months_of_interest.to_frame(index=False, name='Year-Month')

In [ ]:
#This gives us a list of all the "Material Types", and how many checkouts of that type.
materials=df['MaterialType'].value_counts()



In [ ]:
#The output of the above cell is truncated, this lets us see everything.
print(materials.to_string())

In [ ]:
#This functions gives us the number of checkouts of a given material type in a given month
def number_of_checkout(yearmonth,materialtype):
    return df[(df['MaterialType']==materialtype) & (df['date']==yearmonth)].Checkouts.sum()


In [64]:
#I picked these as the materials we cared most about, these are the ones with over 10000000 checkouts
listofmaterialswecareabout=['BOOK','EBOOK','SOUNDDISC','VIDEODISC','AUDIOBOOK']

In [65]:
#This loops over the material we care about and for each of them adds in a column with the number of checkouts in that type in each month
#This code took about an hour to run on my machine.
for materialtype in listofmaterialswecareabout:
    dfmonths[materialtype]= dfmonths.apply(lambda x: number_of_checkout(yearmonth=x['Year-Month'],materialtype=materialtype), axis=1 )

In [ ]:
dfmonths.head()

In [73]:
#This turns the material types into a list that we can iterate over
materialslist=materials.index.tolist()

In [ ]:
#This counts the number of checkouts in a given month of all the data types we want to classify as "other"
#Idea for refactoring: it would be faster to get the total checkouts in a month and then subtract the sum of the types we care most about
def number_of_other_checkout(yearmonth):
    checkouts=0
    for x in materialslist:
        if x not in listofmaterialswecareabout:
            checkouts=checkouts+number_of_checkout(yearmonth,x)
    return checkouts

# number_of_other_checkout('2005-04-01')

In [78]:
#This adds in the "Other" column
# This ran for about 10 hours on my machine 
dfmonths['Other']=dfmonths['Year-Month'].apply(number_of_other_checkout)

In [81]:
dfmonths.to_csv("checkoutstimeseries.csv")